In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import json
import ast

#### load all input data - resume, questionnaire, and follow-up questions. check for null values and duplicates and remove these

In [2]:
intial_resume_df = pd.read_csv('intial_resumes_final.csv').drop('Unnamed: 0',axis=1).reset_index(drop=True)
questionnaires_df = pd.read_csv('questionnaire_5_categories.csv').drop('Unnamed: 0',axis=1).reset_index(drop=True)
followup_df = pd.read_csv('followup_q&a_categorized.csv').drop('Unnamed: 0', axis=1).reset_index(drop=True)
intial_resume_df.head(2) # 1207 resumes

,order_id,parsed_resume,personalInfo,workExp,education,certifications,skills,professionalAffiliations,volunteerExp,awardsAndHonors,professionalSummary
0,5ffc729d0d564400125bcc31,"LORENZO SMITH \n \n \n Ridgefield Park, Ne...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Analyze and build end o...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}","{\n ""professionalSummary"": [\n ""Analyze an..."
1,5ff893a5dd2f9600139ab166,KANESHA JON ES \n 17 Summerfield Place • Nept...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Complaint Management"",\...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}","{\n ""professionalSummary"": [\n ""Quality As..."


In [3]:
intial_resume_df[intial_resume_df.parsed_resume.isnull()] # zero empty resumes

,order_id,parsed_resume,personalInfo,workExp,education,certifications,skills,professionalAffiliations,volunteerExp,awardsAndHonors,professionalSummary


In [4]:
questionnaires_df

,order_id,json_formatted,personalinfo,workexp,education,otherexp,skills
0,5ffc729d0d564400125bcc31,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Lor...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
1,5ff893a5dd2f9600139ab166,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Kan...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
2,5ff8945fdd2f9600139ab16a,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Liz...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
3,5ff7cd94dd2f9600139ab137,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Uma...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
4,5ff7d77cdd2f9600139ab149,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Viv...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
...,...,...,...,...,...,...,...
987,63ab370268746a00137ab45e,"{'1: Contact Information, Needs, Directions': ...",{'1: Did you review and do you understand the ...,"{'1: Employer Name': 'JLM couture ', '2: Brief...","{'1: If you do NOT have a resume, or if you DO...","{""1: Do you have any volunteer experience or a...","{""2: What technical skills do you have that ar..."
988,63c57f42d28912001b92f8fb,{'1: Personal Information and Needs ': {'1: Wh...,{'1: What is your first and last name? ': 'Edw...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar..."
989,63c71397a837f6001a206708,"{'1: Contact Information, Needs, Directions': ...",{'1: Did you review and do you understand the ...,"{'1: Employer Name': ""Philadelphia District At...","{'1: If you do NOT have a resume, or if you DO...","{""1: Do you have any volunteer experience or a...","{""2: What technical skills do you have that ar..."
990,63c57a469cac930013a54719,"{'1: Contact Information, Needs, Directions': ...",{'1: Did you review and do you understand the ...,"{'1: Employer Name': 'Vida', '2: Briefly, what...","{'1: If you do NOT have a resume, or if you DO...","{""1: Do you have any volunteer experience or a...","{""2: What technical skills do you have that ar..."


In [5]:
questionnaires_df[questionnaires_df.json_formatted.isnull()] # zero empty questionnaires
questionnaires_df[questionnaires_df.json_formatted=="""{"1: How to Use the Portal": {"1: Did you watch the video?": "Yes"}}"""] # zero corrupt questionnaires
questionnaires_df = questionnaires_df.drop_duplicates(subset=['order_id'], keep='first').reset_index(drop=True) # around 20 duplicates removed

In [6]:
len(questionnaires_df)

976

In [7]:
followup_df = followup_df[['order_id',
       'personalinfo', 'workexp', 'education', 'otherexp', 'skills']]

In [8]:
# followup_df[followup_df.additional_questionnaire=='{}'] # 271 rows with zero followup questions out of 1004 examples. we will keep these and use it as training examples

In [9]:
followup_df = followup_df.drop_duplicates(subset=['order_id'], keep='first').reset_index(drop=True) # around 13 duplicates removed
len(followup_df)

986

#### merge these 3 input data

In [10]:
import functools as ft
dfs = [intial_resume_df, questionnaires_df, followup_df]

df_input = ft.reduce(lambda left, right: pd.merge(left, right, on='order_id'), dfs)


In [11]:
df_input.columns = ['order_id', 'parsed_resume', 'personalinfo_res', 'workexp_res', 'education_res',
       'certifications_res', 'skills_res', 'professionalaffiliations_res',
       'volunteerexp_res', 'awardsandhonors_res', 'professionalsummary_res',
       'json_formatted_ques', 'personalinfo_ques', 'workexp_ques', 'education_ques',
       'otherexp_ques', 'skills_ques', 'personalinfo_followup','workexp_followup','education_followup','otherexp_followup','skills_followup']

In [12]:
df_input

,order_id,parsed_resume,personalinfo_res,workexp_res,education_res,certifications_res,skills_res,professionalaffiliations_res,volunteerexp_res,awardsandhonors_res,...,personalinfo_ques,workexp_ques,education_ques,otherexp_ques,skills_ques,personalinfo_followup,workexp_followup,education_followup,otherexp_followup,skills_followup
0,5ffc729d0d564400125bcc31,"LORENZO SMITH \n \n \n Ridgefield Park, Ne...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Analyze and build end o...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,{'1: What is your first and last name? ': 'Lor...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar...",['Your current LinkedIn URL is not customized....,['Captivate: Sales Planner - Can you please de...,[],[],"[""Can you please advise if you have experience..."
1,5ff893a5dd2f9600139ab166,KANESHA JON ES \n 17 Summerfield Place • Nept...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Complaint Management"",\...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,{'1: What is your first and last name? ': 'Kan...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar...","['GENERAL: In optimizing your resume, we will ...","['Director, Global Quality Management: What de...",[],[],[]
2,5ff8945fdd2f9600139ab16a,"LIZANN Thompson\t \n 22941 Spicebush Drive, Cl...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Budgeting and cost cont...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award...",...,{'1: What is your first and last name? ': 'Liz...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar...",['1: Your current LinkedIn URL is not customiz...,['Saul Centers Inc - Park Van Ness (General Ma...,"['On your resume, you have that you received y...","['In the questionnaire, you mentioned that you...",[]
3,5ff7cd94dd2f9600139ab137,414-429-4468 | umasenthil2000@gmail.com |http:...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Project Management - Ag...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,{'1: What is your first and last name? ': 'Uma...,{'1: What is the name of your current or most ...,{'1: What is the current or most recent higher...,"{""1: Do you have any volunteer experience rela...","{""1: What technical skills do you have that ar...",['Is there anything not listed on your questio...,"['Senior Programmer Analyst, FIS Global - What...",[],['Just a note: our data-driven research shows ...,[]
4,5ff7d77cdd2f9600139ab149,Vivian N. Ogbolu (Kristine ) \n 2315 Sandi ...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""ski

#### load output data - the draft resume #1

In [13]:
### load in draft resumes (output for resume writer model)
draft_resumes = pd.read_csv('draft_resumes_final.csv').drop(['Unnamed: 0.1','Unnamed: 0','explanations','parsed_resume.1'],axis=1).reset_index(drop=True)
# draft_resumes[draft_resumes.order_id.duplicated()]
draft_resumes

,order_id,parsed_resume,personalInfo,workExp,education,certifications,skills,professionalAffiliations,volunteerExp,awardsAndHonors,professionalSummary,cleaned_certifications,cleaned_professional_affiliations,cleaned_volunteer_experience,cleaned_awards_and_honors,rating,review,createdAt,key,originalname
0,5ff7cd94dd2f9600139ab137,"Uma Senthil \n Menomonee Falls, WI | umasen...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Life Cycle"",\n ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",3.0,Here are some suggestions : \n 1. Add a summar...,2021-01-27 19:54:33.058000+00:00,1611777257216_cf4fad5214e86903a016ce67daa64af1...,Uma Senthil Resume.pdf
1,5ff7cd94dd2f9600139ab137,"Uma Senthil , CSM \n Menomonee Falls, WI | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n\t""certifications"": [{\n\t\t""name"": ""Certif...","{\n ""skills"": [\n ""Product Management"",\n ...","{\n\t""professionalAffiliations"": [{\n\t\t""orga...","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",4.0,NaN,2021-02-15 23:45:48.541000+00:00,1613432726200_ac81624d0e3f7c00bc987a90e7c6d26a...,Uma Senthil Resume V2.pdf
2,5ff7cec9dd2f9600139ab13b,"Janie Delores Johnson \n New Providence, NJ ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""universityN...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [],\n ""technical proficiencies...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}","{\n ""professionalSummary"": [\n ""Direct a t...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",5.0,The resume looks great....I love it.,2021-03-11 12:57:58.398000+00:00,1615467437723_66c79a4cb4e2f1bc626b86a5c2589163...,Janie D. Johnson Resume.pdf
3,5ff7d77cdd2f9600139ab149," KRISTINE OGBOLU -TERRELL \n Sachse, TX Z...","{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n\t""workExperience"": [{\n\t\t\t""companyName""...","{\n ""education"": [\n {\n ""universityN...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n ""professionalAffiliations"": [\n ...","{\n ""volunteerExperiences"": [\n {\n ""...","{\n ""awardsAndHonors"": [\n {\n ...","{\n ""professionalSummary"": [\n ""Highly mot...","[\n {\n ""name"": ""Annual Banking Compliance...","[\n {\n ""organizationName"": ""Texas Associa...","[\n {\n ""organizationName"": ""Children’s Mi...","[\n {\n ""awardName"": ""Five Years of servic...",5.0,NaN,2021-01-21 15:18:00.081000+00:00,1611242273050_7d83d1507215fa12bccfbf34ce39491c...,Kristine_Ogbolu-Terrell_Resume_AFTER.pdf
4,5ff7d77cdd2f9600139ab149,"KRISTINE OGBOLU -TERRELL \n Sachse, TX 7504...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": [\n {\n ""...","{\n ""awardsAndHonors"": [\n {\n 

In [14]:
draft_resumes.columns

Index(['order_id', 'parsed_resume', 'personalInfo', 'workExp', 'education',
       'certifications', 'skills', 'professionalAffiliations', 'volunteerExp',
       'awardsAndHonors', 'professionalSummary', 'cleaned_certifications',
       'cleaned_professional_affiliations', 'cleaned_volunteer_experience',
       'cleaned_awards_and_honors', 'rating', 'review', 'createdAt', 'key',
       'originalname'],
      dtype='object')

In [15]:
draft_resumes = draft_resumes[['order_id', 'parsed_resume', 'personalInfo', 'workExp', 'education', 'cleaned_certifications',
        'skills',  'professionalSummary', 
       'cleaned_professional_affiliations', 'cleaned_volunteer_experience',
       'cleaned_awards_and_honors', 'createdAt']]
draft_resumes.columns = ['order_id', 'parsed_resume', 'personalInfo', 'workExp', 'education', 'certifications',
        'skills',  'professionalSummary', 
       'professional_affiliations', 'volunteer_experience',
       'awards_and_honors', 'createdAt']
draft_resumes

,order_id,parsed_resume,personalInfo,workExp,education,certifications,skills,professionalSummary,professional_affiliations,volunteer_experience,awards_and_honors,createdAt
0,5ff7cd94dd2f9600139ab137,"Uma Senthil \n Menomonee Falls, WI | umasen...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Life Cycle"",\n ...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",2021-01-27 19:54:33.058000+00:00
1,5ff7cd94dd2f9600139ab137,"Uma Senthil , CSM \n Menomonee Falls, WI | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Management"",\n ...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",2021-02-15 23:45:48.541000+00:00
2,5ff7cec9dd2f9600139ab13b,"Janie Delores Johnson \n New Providence, NJ ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""universityN...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [],\n ""technical proficiencies...","{\n ""professionalSummary"": [\n ""Direct a t...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",2021-03-11 12:57:58.398000+00:00
3,5ff7d77cdd2f9600139ab149," KRISTINE OGBOLU -TERRELL \n Sachse, TX Z...","{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n\t""workExperience"": [{\n\t\t\t""companyName""...","{\n ""education"": [\n {\n ""universityN...","[\n {\n ""name"": ""Annual Banking Compliance...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n ""professionalSummary"": [\n ""Highly mot...","[\n {\n ""organizationName"": ""Texas Associa...","[\n {\n ""organizationName"": ""Children’s Mi...","[\n {\n ""awardName"": ""Five Years of servic...",2021-01-21 15:18:00.081000+00:00
4,5ff7d77cdd2f9600139ab149,"KRISTINE OGBOLU -TERRELL \n Sachse, TX 7504...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""explanations"": {\n ""Licensed Real Est...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n\t""professionalSummary"": [""Highly motivated...","{\n ""explanations"": {\n ""Texas Association...","{\n ""explanations"": {\n ""Children’s Minist...","{\n ""explanations"": {\n ""Star Scene Award ...",2021-02-09 15:57:10.227000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2305,63c57a469cac930013a54719,"Geoffrey Lee \n River Falls, Wisconsin | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Cloud Architecture & Developm...","{\n ""professionalSummary"": [\n ""Attention ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",2023-01-23 16:44:29.481000+00:00
2306,63c57f42d28912001b92f8fb,"Edward Lowe \n Waco, TX | (254) 651 -5600 | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Sales Development"", ""Account ...","{\n ""professionalSummary"": [\n ""Self-motiv...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award...",2023-01-20 10:54:05.579000+00:00
2307,63c71397a837f6001a206708,"W

##### get first draft 

In [16]:
draft_resumes["createdAt"] = pd.to_datetime(draft_resumes["createdAt"],format='mixed')
tmp_df = draft_resumes.sort_values(by=['order_id', 'createdAt']).groupby('order_id')['createdAt'].first() # for each order id, get first draft
tmp_df

/var/folders/mg/b267j_yn6js7myvq7tnf6_v40000gn/T/ipykernel_67927/2632935038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_resumes["createdAt"] = pd.to_datetime(draft_resumes["createdAt"],format='mixed')


order_id
5ff7cd94dd2f9600139ab137   2021-01-27 19:54:33.058000+00:00
5ff7cec9dd2f9600139ab13b   2021-03-11 12:57:58.398000+00:00
5ff7d77cdd2f9600139ab149   2021-01-21 15:18:00.081000+00:00
5ff893a5dd2f9600139ab166   2021-01-14 09:27:46.858000+00:00
5ff8945fdd2f9600139ab16a   2021-01-16 04:58:22.016000+00:00
                                         ...               
63c57a469cac930013a54719   2023-01-23 16:44:29.481000+00:00
63c57f42d28912001b92f8fb   2023-01-20 10:54:05.579000+00:00
63c71397a837f6001a206708   2023-02-10 19:51:01.860000+00:00
63c993bf1ce2490014d3abb9   2023-01-26 09:45:16.878000+00:00
63c99ddaa837f6001a239c0a   2023-01-28 14:47:34.016000+00:00
Name: createdAt, Length: 1145, dtype: datetime64[ns, UTC]

In [17]:
df_output = pd.merge(tmp_df,draft_resumes,on=['order_id','createdAt'])
df_output # 1145 1st draft resumes

,order_id,createdAt,parsed_resume,personalInfo,workExp,education,certifications,skills,professionalSummary,professional_affiliations,volunteer_experience,awards_and_honors
0,5ff7cd94dd2f9600139ab137,2021-01-27 19:54:33.058000+00:00,"Uma Senthil \n Menomonee Falls, WI | umasen...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Life Cycle"",\n ...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
1,5ff7cec9dd2f9600139ab13b,2021-03-11 12:57:58.398000+00:00,"Janie Delores Johnson \n New Providence, NJ ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""universityN...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [],\n ""technical proficiencies...","{\n ""professionalSummary"": [\n ""Direct a t...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
2,5ff7d77cdd2f9600139ab149,2021-01-21 15:18:00.081000+00:00," KRISTINE OGBOLU -TERRELL \n Sachse, TX Z...","{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n\t""workExperience"": [{\n\t\t\t""companyName""...","{\n ""education"": [\n {\n ""universityN...","[\n {\n ""name"": ""Annual Banking Compliance...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n ""professionalSummary"": [\n ""Highly mot...","[\n {\n ""organizationName"": ""Texas Associa...","[\n {\n ""organizationName"": ""Children’s Mi...","[\n {\n ""awardName"": ""Five Years of servic..."
3,5ff893a5dd2f9600139ab166,2021-01-14 09:27:46.858000+00:00,"Page 1 of 2 KANESHA JONES \n Neptune, NJ  ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ...","{\n\t""education"": [{\n\t\t\t""universityName"": ...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Quality Control (QC)"",\...","{\n ""professionalSummary"": [\n ""Quality-co...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": [],\n ""explanati...","{\n ""awardsAndHonors"": [],\n ""explanation"": ..."
4,5ff8945fdd2f9600139ab16a,2021-01-16 04:58:22.016000+00:00,"Lizann Thompson \n Clarksburg , MD  lizanno...","{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Account Management"", ""Custome...","{\n ""professionalSummary"": [\n ""Spearhead ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
...,...,...,...,...,...,...,...,...,...,...,...,...
1140,63c57a469cac930013a54719,2023-01-23 16:44:29.481000+00:00,"Geoffrey Lee \n River Falls, Wisconsin | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Cloud Architecture & Developm...","{\n ""professionalSummary"": [\n ""Attention ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
1141,63c57f42d28912001b92f8fb,2023-01-20 10:54:05.579000+00:00,"Edward Lowe \n Waco, TX | (254) 651 -5600 | ...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Sales Development"", ""Account ...","{\n ""professionalSummary"": [\n ""Self-motiv...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award..."
1142,63c71397a837f6001a206708,2023-02-10 19

#### merge these input-output examples

In [18]:
# there are 976 inputs and # 1145 outputs

In [19]:
df_output.columns

Index(['order_id', 'createdAt', 'parsed_resume', 'personalInfo', 'workExp',
       'education', 'certifications', 'skills', 'professionalSummary',
       'professional_affiliations', 'volunteer_experience',
       'awards_and_honors'],
      dtype='object')

In [20]:
df_output = df_output[['order_id', 'personalInfo', 'workExp',
       'education', 'certifications', 'skills', 'professionalSummary',
       'professional_affiliations', 'volunteer_experience',
       'awards_and_honors']]
df_output.columns = ['order_id', 'personalInfo_draft', 'workExp_draft',
       'education_draft', 'certifications_draft', 'skills_draft', 'professionalSummary_draft',
       'professional_affiliations_draft', 'volunteer_experience_draft',
       'awards_and_honors_draft']

In [21]:
df_output

,order_id,personalInfo_draft,workExp_draft,education_draft,certifications_draft,skills_draft,professionalSummary_draft,professional_affiliations_draft,volunteer_experience_draft,awards_and_honors_draft
0,5ff7cd94dd2f9600139ab137,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Life Cycle"",\n ...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
1,5ff7cec9dd2f9600139ab13b,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""universityN...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [],\n ""technical proficiencies...","{\n ""professionalSummary"": [\n ""Direct a t...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
2,5ff7d77cdd2f9600139ab149,"{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n\t""workExperience"": [{\n\t\t\t""companyName""...","{\n ""education"": [\n {\n ""universityN...","[\n {\n ""name"": ""Annual Banking Compliance...","{\n ""skills"": [\n ""Financial Analysis"",\n ...","{\n ""professionalSummary"": [\n ""Highly mot...","[\n {\n ""organizationName"": ""Texas Associa...","[\n {\n ""organizationName"": ""Children’s Mi...","[\n {\n ""awardName"": ""Five Years of servic..."
3,5ff893a5dd2f9600139ab166,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ...","{\n\t""education"": [{\n\t\t\t""universityName"": ...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Quality Control (QC)"",\...","{\n ""professionalSummary"": [\n ""Quality-co...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": [],\n ""explanati...","{\n ""awardsAndHonors"": [],\n ""explanation"": ..."
4,5ff8945fdd2f9600139ab16a,"{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Account Management"", ""Custome...","{\n ""professionalSummary"": [\n ""Spearhead ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
...,...,...,...,...,...,...,...,...,...,...
1140,63c57a469cac930013a54719,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Cloud Architecture & Developm...","{\n ""professionalSummary"": [\n ""Attention ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
1141,63c57f42d28912001b92f8fb,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Sales Development"", ""Account ...","{\n ""professionalSummary"": [\n ""Self-motiv...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award..."
1142,63c71397a837f6001a206708,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Career Prosecutor"",\n ...","{\n ""professionalSummary"": [""Dedicated and pe...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award..."
1143,63c993bf1ce2490014d3abb9,"{\n ""personalInformation"": {\n ""firstName""...","{\n\t""workExperience"": [{\n\t\t\t""companyName""...","{\n ""

In [ ]:
#### input - output combine

In [106]:
input_output_df = df_input.merge(df_output, on='order_id',how='inner').reset_index(drop=True)
# input_output_df.to_csv('intial_to_draft_train.csv')
input_output_df

,order_id,parsed_resume,personalinfo_res,workexp_res,education_res,certifications_res,skills_res,professionalaffiliations_res,volunteerexp_res,awardsandhonors_res,...,skills_followup,personalInfo_draft,workExp_draft,education_draft,certifications_draft,skills_draft,professionalSummary_draft,professional_affiliations_draft,volunteer_experience_draft,awards_and_honors_draft
0,5ffc729d0d564400125bcc31,"LORENZO SMITH \n \n \n Ridgefield Park, Ne...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Analyze and build end o...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,"[""Can you please advise if you have experience...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [""Sales Planning"", ""Media Plann...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
1,5ff893a5dd2f9600139ab166,KANESHA JON ES \n 17 Summerfield Place • Nept...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Complaint Management"",\...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,[],"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ...","{\n\t""education"": [{\n\t\t\t""universityName"": ...","{\n ""certifications"": []\n}","{\n ""skills"": [\n ""Quality Control (QC)"",\...","{\n ""professionalSummary"": [\n ""Quality-co...","{\n ""professionalAffiliations"": [\n {\n ...","{\n ""volunteerExperiences"": [],\n ""explanati...","{\n ""awardsAndHonors"": [],\n ""explanation"": ..."
2,5ff8945fdd2f9600139ab16a,"LIZANN Thompson\t \n 22941 Spicebush Drive, Cl...","{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Budgeting and cost cont...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": [\n {\n ""award...",...,[],"{\n\t""personalInformation"": {\n\t\t""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [""Account Management"", ""Custome...","{\n ""professionalSummary"": [\n ""Spearhead ...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
3,5ff7cd94dd2f9600139ab137,414-429-4468 | umasenthil2000@gmail.com |http:...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Project Management - Ag...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}",...,[],"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n ""education"": [\n {\n ""univerityNa...","{\n ""certifications"": [\n {\n ""name"":...","{\n ""skills"": [\n ""Product Life Cycle"",\n ...","{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalAffiliations"": []\n}","{\n ""volunteerExperiences"": []\n}","{\n ""awardsAndHonors"": []\n}"
4,5ff7d77cdd2f9600139ab149,Vivian N. Ogbolu (Kristine ) \n 2315 Sandi ...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""workExperience"": [\n {\n ""compan...","{\n

#### create training data (input-output examples) for each of the 8 sections. this includes the prompts that will be fed to the model to "write the draft (output)"

In [66]:
### we need to provide career goals and work experience as input to the model for it to write work experience section
write_workexp_section = """You are given the following information about the candidate’s work experience, sourced from an existing resume, career questionnaire, and certain follow-up questions and answers. Your task is to draft a targeted, specific work experience section using the provided information. You will be rewarded for following this guideline: 
1. Completeness: Each job experience should include the candidate's title, company name, location of work, and dates of employment with both months and years. 
2. CAR Transform: For each job experience, transform the accomplishments and responsibilities into a list of bullets. Each bullet will present an accomplishment or responsibility using the CAR (Challenge, Action, Result) format. Each bullet will include a description of the task or challenge faced, the action that the candidate took to address the challenge, and a result achieved in the form of a metric. When a quantifiable result is not available, you will provide a qualitative result. To do this, follow this instruction: 
2.1 Review the CAR statements provided in the follow-up questions and answers. They may be written like this from the customer’s point of view: 
Challenge: I needed to recoup a revenue loss after a major partner canceled their partnership
Action: I scoured my network and discovered a new source of revenue that was a previously un-tapped market of 3,000
Result: I implemented a new service in 30 days. We had a 50% increase in revenue over the prior month
Your job is to convert the CAR statement into an expertly-crafted resume bullet. You do this by reversing the structure of the CAR statement so that you present the new statement starting with the result, simplifying and streamlining the language. An example would be: “Implemented a new service that reached 3,000 new customers in 30 days, marking a 50% increase in revenue, even after the loss of a key client.”
2.2 Sometimes there isn’t enough information for a CAR statement. This is usually the case for job duties and responsibilities completed regularly - as opposed to accomplishments that would be described in a CAR statement. Include duties and responsibilities after the accomplishment bullets. Infer some generic duties and responsibilities based on job titles if this is the case. Include keywords from the job descriptions the customer is interested in as long as it is clear the customer likely did those things. You will be rewarded for making it highly tailored to the desired job title/level.
2.3 There should be at least 2 bullets per job experience, but an ideal number is 3-7 bullets per job experience. Do not exceed 10 bullets per job experience.
3. Grammar: The resulting bullet points should be 1 sentence and should not include adverbs. Specifically, never use the word "successfully" because it is a redundant term. It should be written in third-person POV but not use pronouns. It should not use articles like “the” or “a” either. Start every bullet with a strong action verb. Avoid the verbs "helped", "assisted", and "supported". Use strong action words: start sentences with impactful verbs like "Led", "Developed", "Implemented", etc., to convey confidence and initiative. Do not start bullets with the same verb.

Finally return a JSON object (follow the resume's JSON schema) with a "work experience" field containing your transformed results. You will be rewarded for following the instructions above without omitting any work experience information from the sources given to you. 

Here's information from resume: 
{workexp_res}

Here's information from career questionnaire: 
{personalinfo_ques}
{workexp_ques}

Here's information from follow-up questions and answers asked to the candidate: 
{personalinfo_followup}
{workexp_followup}

"""

### we need to provide career goals, work experience, and skills as input
write_skills_section = """You are given the following information about a candidate, sourced from an existing resume, career questionnaire, and certain follow-up questions and answers. Your job is to craft the skills section of their resume using this information. A skills section on a resume highlights the client’s abilities and expertise that align with the jobs they are targeting. You will not just list all skills mentioned in the existing resume, career questionnaire, and follow up questions and answers. You will create a list of skills that demonstrate the client’s ability to do the job they are hoping to get hired for (this information can be found under questions related to career goals). Follow these guidelines:

1. Use your knowledge of the candidate's desired job titles and job descriptions to identify highly relevant keywords and high-impact skills.
2. Examine the candidate's profile across all sources to confirm if they have demonstrated these skills in their professional career before including them. Do not include skills the client does not possess. 
3. Limit the list to no more than 20 skills.
4. Avoid soft skills and cliches such as "Communication" or "Problem Solving" unless it is a primary requirement of the job the candidate wants.

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the resume's JSON schema) with a "skills" field containing your transformed results.

Here's information from resume: 
{workexp_res}
{skills_res}

Here's information from career questionnaire: 
{personalinfo_ques}
{workexp_ques}
{skills_ques}

Here's information from follow-up questions and answers asked to the candidate: 
{personalinfo_followup}
{workexp_followup}
{skills_followup}

"""

### we need to provide education only as input (from resume, ques, followup)
write_education_section = """You are given the following information about the candidate, sourced from an existing resume, career questionnaire, and certain follow-up questions and answers. Your job is to craft the education section of their resume using this information. Follow this guideline: 
1. Each education experience should include the following specific data elements: degree or qualification title, school or university name, date of graduation (including month and year). The client might have more than one education experience. If the client has multiple degrees from one school or university, the degrees and date of graduation can be listed under one school or university name. 
2. Do not include a GED or High School Diploma even if it is the highest education attained unless the candidate is still attending school.
3. Omit graduation dates if they are more than 7 years old to prevent age bias.

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the resume's JSON schema) with a "education" field containing your transformed results.

Here's information from resume: 
{education_res}

Here's information from career questionnaire: 
{education_ques}

Here's information from follow-up questions and answers asked to the candidate: 
{education_followup}

"""

### we provide career goals and "other experience" as input (since it contains cert info)
write_certifications_section = """You are provided with the following information about a candidate's "certifications", gathered from their existing resume, a career questionnaire, and responses to follow-up questions. "Certifications" are formal credentials granted after completing specific training or passing exams to validate professional skills and knowledge. It does not refer to "awards and honors" which are recognitions and accolades received for exceptional achievements, performance, or contributions in academic, professional, or extracurricular activities. Your task is to determine which of these certifications are relevant to the candidate's career goals and the job they are applying to and craft the certifications section of the candidate's resume. Each certification should include the name of the certification, certification date, certification number, and issuing authority. 

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the schema below) with a "certifications" field containing your answer.
{
  "certifications": [
    {
      "name": "string",
      "expectedCertificationDate": "date",
      "certificationNumber": "string",
      "issuingAuthority": "string"  
    }
  ]
}

Here's information from resume: 
{certifications_res}

Here's information from questionnaire: 
{otherexp_ques}
{personalinfo_ques}

Here's information from follow-up questions and answers asked to the candidate:: 
{otherexp_followup}
{personalinfo_followup}

"""

### we provide career goals and "other experience" as input (since it contains volunteer exp info)
write_volunteer_section = """You are provided with the following information about a candidate's "volunteering experiences", gathered from their existing resume, a career questionnaire, and responses to follow-up questions. "Volunteering Experiences" involve unpaid work done for community service, charitable organizations, or non-profits. This does not include experiences that are part of professional work experience. Your task is to determine which of these volunteering experiences are relevant to the candidate's career goals and the job they are applying to and craft the "Volunteering Experiences" section of the candidate's resume. Each experience should include the organization name, role, and date.

Key points to remember:
1. Earlier Experience: Experiences older than 15 years generally don’t need to be mentioned on the resume. Notable exceptions are when the customer wants to include older experiences because of the company they worked for perhaps representing a certain element of their background they may not otherwise get across from their current experience. For example: to name-drop a well-known company like Disney as a prior employer, or to emphasize work completed at a startup company over 15 years ago if all of their current experience is from traditional corporate environments. No need to include details like descriptions or accomplishments for early experiences.
2. Other Sections: Ensure relevance to the candidate’s current goals. For example, a Software Engineering role typically doesn’t require volunteer experience at an animal shelter. However, a recent business professional might include a college debate championship under Awards, whereas someone with 10 years of experience might not need to list early awards due to more recent and relevant achievements.

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the schema below) with a "Volunteering Experiences" field containing your answer.
{
  "volunteerExperiences": [ 
    {
      "organizationName": "string",
      "role": "string",
      "date": "date"
    }
  ]
}

Here's information from resume: 
{volunteer_experience_res}

Here's information from questionnaire: 
{otherexp_ques}
{personalinfo_ques}

Here's information from follow-up questions and answers asked to the candidate:: 
{otherexp_followup}
{personalinfo_followup}

"""


### we provide career goals and "other experience" as input (since it contains awards and honors info)
write_awards_section = """You are provided with the following information about a candidate's "awards and honors", gathered from their existing resume, a career questionnaire, and responses to follow-up questions. "Awards and Honors" refer to recognitions and accolades received for exceptional achievements, performance, or contributions in academic, professional, or extracurricular activities. It does not refer to "certifications" which are formal credentials granted after completing specific training or passing exams to validate professional skills and knowledge. Your task is to determine which of these awards are relevant to the candidate's career goals and the job they are applying to and craft the awards and honors section of the candidate's resume. Each award should include the name of the award, awarded date, and issuing authority. 

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the schema below) with a "awards and honors" field containing your answer.
{
  "awardsAndHonors": [
    {
      "awardName": "string",
      "organization": "string",
      "date": "date"   
    } 
  ]
}

Here's information from resume: 
{awards_and_honors_res}

Here's information from questionnaire: 
{otherexp_ques}
{personalinfo_ques}

Here's information from follow-up questions and answers asked to the candidate:: 
{otherexp_followup}
{personalinfo_followup}

"""

### we provide career goals and "other experience" as input (since it contains prof affiliations info)
write_affiliations_section = """You are provided with the following information about a candidate's "professional affiliations", gathered from their existing resume, a career questionnaire, and responses to follow-up questions. "Professional Affiliations" refer to memberships or active participation in professional organizations, associations, or societies relevant to one's career or industry. This does not include experiences that are part of professional work experience neither does it include work that counts towards volunteering experiences (unpaid work done for community service, charitable organizations, or non-profits). Your task is to determine which of these affiliations are relevant to the candidate's career goals and the job they are applying to and craft the professional affiliations section of the candidate's resume. Each affiliation should include the organization name, role, and date. 

Key points to remember:
1. Earlier Experience: Experiences older than 15 years generally don’t need to be mentioned on the resume. Notable exceptions are when the customer wants to include older experiences because of the company they worked for perhaps representing a certain element of their background they may not otherwise get across from their current experience. For example: to name-drop a well-known company like Disney as a prior employer, or to emphasize work completed at a startup company over 15 years ago if all of their current experience is from traditional corporate environments. No need to include details like descriptions or accomplishments for early experiences.

You will be rewarded for carefully following the above guideline and not omitting any of the information given to you.

Finally return a JSON object (follow the schema below) with a "professional affiliations" field containing your answer.
{
  "professionalAffiliations": [ 
    {
      "organizationName": "string",
      "role": "string",
      "date": "date"
    }
  ]
}

Here's information from resume: 
{professional_affiliations_res}

Here's information from questionnaire: 
{otherexp_ques}
{personalinfo_ques}

Here's information from follow-up questions and answers asked to the candidate:: 
{otherexp_followup}
{personalinfo_followup}

"""

#### professional summary is written at the end of the process. therefore, feed it draft resume sections as input. 
### we need to give as input work experience, skills, and education. also career goals.
write_professionalSummary_section = """You are given the following information about the candidate, sourced from an existing resume. Your task is to use the candidate’s work experience, skills, and education details to craft an excellent professional summary. This summary should be tailored to match the desired position that the candidate wishes to apply for. This section is crucial as it sets the tone for the entire resume, capturing the essence of a candidate’s experience and goals. Aim for 3-4 sentences because the summary should be a snapshot, not a detailed biography. Highlight key achievements: focus on significant accomplishments and skills that align with the client’s goals. Use quantifiable metrics whenever possible. Use strong action words: Start sentences with impactful verbs like "Led", "Developed", "Implemented", etc., to convey confidence and initiative. Highlight any specialized skills or expertise that make the candidate stand out from the competition.

You will be rewarded for carefully following the above-mentioned points. 

Finally, return a JSON object with a "professional summary" field containing your result.

Here's information from resume: 
{workexp_draft}
{skills_draft}
{education_draft}

Here's information on their career goals: 
{personalinfo_ques}

"""

### only feed it personal info from all 3 sources
write_personalInfo_section = """You are given the following personal information about the candidate, sourced from an existing resume, career questionnaire, and certain follow-up questions and answers. Your job is to craft the "personal information" section of their resume using this information. 

Finally return a JSON object (follow the schema below) with a "personal information" field containing your transformed results.
{
  "personalInformation": {
    "firstName": "string",
    "lastName": "string",
    "phoneNumber": "string",
    "email": "email",
    "location": {
      "state": "string",
      "country": "string"
    },
    "linkedinURL": "url",
    "professionalWebsites": ["url"]
  }
}

Here's information from resume: 
{personalinfo_res}

Here's information from questionnaire: 
{personalinfo_ques}

Here's information from follow-up questions and answers asked to the candidate:: 
{personalinfo_followup}

"""


In [127]:
input_output_df.columns

Index(['Unnamed: 0', 'order_id', 'parsed_resume', 'personalinfo_res',
       'workexp_res', 'education_res', 'certifications_res', 'skills_res',
       'professionalaffiliations_res', 'volunteerexp_res',
       'awardsandhonors_res', 'professionalsummary_res', 'json_formatted_ques',
       'personalinfo_ques', 'workexp_ques', 'education_ques', 'otherexp_ques',
       'skills_ques', 'personalinfo_followup', 'workexp_followup',
       'education_followup', 'otherexp_followup', 'skills_followup',
       'personalInfo_draft', 'workExp_draft', 'education_draft',
       'certifications_draft', 'skills_draft', 'professionalSummary_draft',
       'professional_affiliations_draft', 'volunteer_experience_draft',
       'awards_and_honors_draft'],
      dtype='object')

In [128]:
def _get_model_output(instruction):
    client =  OpenAI(
    api_key="",
)
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [
        {"role": "system", "content": "You are a highly trained resume writer with extensive expertise in creating professional resumes that effectively highlight clients' skills, experiences, and qualifications. Your role involves meticulously crafting each section of the resume to ensure clarity, impact, and relevance to the desired job role. You will use the CAR (Challenge, Action, Result) format to write bullets for each work experience the candidate has. The result will be a list of bullets under each work experience. These bullets will be the client’s key achievements and show the candidate as capable and qualified. The bullets should not be a basic list of job duties. If the user provides feedback on the draft resume, you will incorporate the feedback and produce revised versions until the user is satisfied with the final resume."},
        {"role": "user", "content": instruction}
    ], 
    response_format={"type": "json_object"}
            )
    result = response.choices[0].message.content
    return result

In [130]:
instr = []
sys = []
cho = []
rej = []
for id, row in input_output_df.iterrows():
    workexp_draft = row['workExp_draft']
    skills_draft = row['skills_draft']
    education_draft = row['education_draft']

    personalinfo_ques = row['personalinfo_ques']

    instruction = f"""You are given the following information about the candidate, sourced from an existing resume. Your task is to use the candidate’s work experience, skills, and education details to craft an excellent professional summary. This summary should be tailored to match the desired position that the candidate wishes to apply for. This section is crucial as it sets the tone for the entire resume, capturing the essence of a candidate’s experience and goals. Aim for 3-4 sentences because the summary should be a snapshot, not a detailed biography. Highlight key achievements: focus on significant accomplishments and skills that align with the client’s goals. Use quantifiable metrics whenever possible. Use strong action words: Start sentences with impactful verbs like "Led", "Developed", "Implemented", etc., to convey confidence and initiative. Highlight any specialized skills or expertise that make the candidate stand out from the competition.

You will be rewarded for carefully following the above-mentioned points. 

Finally, return a JSON object with a "professionalSummary" field containing your result.

Here's information from resume: 
{workexp_draft}
{skills_draft}
{education_draft}

Here's information on their career goals: 
{personalinfo_ques}

"""
    
    system = instruction
    chosen = row['professionalSummary_draft']
    rejected = _get_model_output(instruction)
    print (id, rejected)

    instr.append(instruction)
    sys.append(system)
    cho.append(chosen)
    rej.append(rejected)
   
train_df = pd.DataFrame()
train_df['order_id'] = input_output_df['order_id']
train_df['system'] = sys
train_df['instruction'] = instr
train_df['chosen'] = cho
train_df['rejected'] = rej

0 {
  "professionalSummary": "Dynamic and results-driven Sales Planner with a proven track record of creating impactful media plans to achieve client KPIs in the DOOH advertising space. Skilled in collaborating with account executives to generate comprehensive end-of-campaign reports and leveraging advanced Excel functions for data analysis. Led successful programmatic advertising initiatives as a Manager, driving business growth and spearheading strategic proposals. Adept at identifying target audiences and utilizing DSPs to optimize campaign performance. Seeking to leverage expertise in media planning and sales coordination to excel as an Account Executive in the Digital Media industry."
}
1 {
    "professionalSummary": "Experienced and results-driven Director of Global Quality Management specializing in Combination Products with a track record of enhancing site quality metrics, driving compliance with GMP/FDA regulations, and leading cross-functional teams to ensure product quality 

In [131]:
train_df

,order_id,system,instruction,chosen,rejected
0,5ffc729d0d564400125bcc31,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Dynamic and resul..."
1,5ff893a5dd2f9600139ab166,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Quality-co...","{\n ""professionalSummary"": ""Experienced and..."
2,5ff8945fdd2f9600139ab16a,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Spearhead ...","{\n ""professionalSummary"": ""Dynamic and resul..."
3,5ff7cd94dd2f9600139ab137,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Results-driven Se..."
4,5ff7d77cdd2f9600139ab149,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Highly mot...","{\n\t""professionalSummary"": ""Experienced banki..."
...,...,...,...,...,...
851,63ab370268746a00137ab45e,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Innovative and de..."
852,63c57f42d28912001b92f8fb,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Self-motiv...","{\n ""professionalSummary"": ""Edward Lowe is a ..."
853,63c71397a837f6001a206708,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [""Dedicated and pe...","{\n ""professionalSummary"": ""Dedicated Career ..."
854,63c57a469cac930013a54719,You are given the following information about ...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Attention ...","{\n ""professionalSummary"": ""Dedicated Senior ..."


##### unsloth requires slightly different data format

In [132]:
# train_df = train_df[train_df.chosen.notnull()]

In [133]:
train = train_df.copy()

instr = []
inp = []
cho = []
rej = []
for id, row in train.iterrows():
    input_ = row['system']

    instruction = "You are a highly trained resume writer with extensive expertise in creating professional resumes that effectively highlight clients' skills, experiences, and qualifications. Your role involves meticulously crafting each section of the resume to ensure clarity, impact, and relevance to the desired job role. You will use the CAR (Challenge, Action, Result) format to write bullets for each work experience the candidate has. The result will be a list of bullets under each work experience. These bullets will be the client’s key achievements and show the candidate as capable and qualified. The bullets should not be a basic list of job duties. If the user provides feedback on the draft resume, you will incorporate the feedback and produce revised versions until the user is satisfied with the final resume."

    chosen = row['chosen']
    rejected = row['rejected']

    instr.append(instruction)
    inp.append(input_)
    cho.append(chosen)
    rej.append(rejected)
   
train['instruction'] = instruction
train['input'] = inp
train['chosen'] = cho
train['rejected'] = rej

In [134]:
train = train[['order_id','instruction','input','chosen','rejected']]
train

,order_id,instruction,input,chosen,rejected
0,5ffc729d0d564400125bcc31,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Dynamic and resul..."
1,5ff893a5dd2f9600139ab166,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Quality-co...","{\n ""professionalSummary"": ""Experienced and..."
2,5ff8945fdd2f9600139ab16a,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Spearhead ...","{\n ""professionalSummary"": ""Dynamic and resul..."
3,5ff7cd94dd2f9600139ab137,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Results-driven Se..."
4,5ff7d77cdd2f9600139ab149,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Highly mot...","{\n\t""professionalSummary"": ""Experienced banki..."
...,...,...,...,...,...
851,63ab370268746a00137ab45e,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Experience...","{\n ""professionalSummary"": ""Innovative and de..."
852,63c57f42d28912001b92f8fb,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Self-motiv...","{\n ""professionalSummary"": ""Edward Lowe is a ..."
853,63c71397a837f6001a206708,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [""Dedicated and pe...","{\n ""professionalSummary"": ""Dedicated Career ..."
854,63c57a469cac930013a54719,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""professionalSummary"": [\n ""Attention ...","{\n ""professionalSummary"": ""Dedicated Senior ..."


In [135]:
# train[train.chosen.isnull()]

,order_id,instruction,input,chosen,rejected


In [136]:
train.to_csv('profSummary_write_train.csv')

#### combine all 9 train datasets into one for orpo mix training


In [140]:
df1 = pd.read_csv('train_datasets/workexp_write_train.csv')
df2 = pd.read_csv('train_datasets/certifications_write_train.csv')
df3 = pd.read_csv('train_datasets/skills_write_train.csv')
df4 = pd.read_csv('train_datasets/personalinfo_write_train.csv')
df5 = pd.read_csv('train_datasets/awardsandhonors_write_train.csv')
df6 = pd.read_csv('train_datasets/volunteerexp_write_train.csv')
df7 = pd.read_csv('train_datasets/education_write_train.csv')
df8 = pd.read_csv('train_datasets/profaffiliations_write_train.csv')
df9 = pd.read_csv('train_datasets/profSummary_write_train.csv')

train_all = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9],ignore_index=True).sample(frac=1).reset_index(drop=True)
train_all = train_all.drop('Unnamed: 0',axis=1)
train_all = train_all.drop('Unnamed: 0.1',axis=1)
train_all

,order_id,instruction,input,chosen,rejected
0,632dcb355bc4b900149aea9c,You are a highly trained resume writer with ex...,You are provided with the following informatio...,"{\n ""professionalAffiliations"": []\n}","{\n ""professionalAffiliations"": [\n {\n ..."
1,60008636b364050012c3e565,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n\t""workExperience"": [{\n\t\t""companyName"": ...","{\n ""workExperience"": [\n {\n ..."
2,6052773e40c7c300125e8aeb,You are a highly trained resume writer with ex...,You are provided with the following informatio...,"{\n ""awardsAndHonors"": [\n {\n ""award...","{\n ""awardsAndHonors"": [\n {\n ""award..."
3,60a8e5f1c8a376001303f150,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""skills"": [\n ""Director of Operations""...","{\n\t""skills"": [""Project Management"", ""Operati..."
4,624afff0bae8cd001ba9c067,You are a highly trained resume writer with ex...,You are provided with the following informatio...,"{\n ""volunteerExperiences"": []\n}","{\n ""volunteerExperiences"": []\n}"
...,...,...,...,...,...
7699,60a6921ec8a376001303f0fb,You are a highly trained resume writer with ex...,You are given the following information about ...,"{\n ""education"": [\n {\n ""universityN...","{\n ""education"": [\n {\n ""degreeTitle..."
7700,60d34f2a563922001937bfa6,You are a highly trained resume writer with ex...,You are provided with the following informatio...,"{\n ""certifications"": []\n}","{\n ""certifications"": [\n {\n ""name"":..."
7701,620a750f4904870015fd9923,You are a highly trained resume writer with ex...,You are provided with the following informatio...,"{\n ""explanation"": ""The current section for p...","{\n ""professionalAffiliations"": []\n}"
7702,63179439e1f5720013080d3d,You are a highly trained resume writer with ex...,You are given the following personal informati...,"{\n ""personalInformation"": {\n ""firstName""...","{\n ""personalInformation"": {\n ""firstName""..."


In [145]:
train_all.to_csv('write_train_orpo_mix.csv')